In [1]:
!ls

BMEhw2.ipynb  experiments  MedMNIST2D  ondemand  Untitled.ipynb


In [3]:
# Define the device you want to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
import torch
import os
import pandas as pd

In [6]:
# Install relevant dependencies
!pip install git+https://github.com/M3DV/ACSConv.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/M3DV/ACSConv.git to /tmp/pip-req-build-kmv24qus
  Running command git clone --filter=blob:none --quiet https://github.com/M3DV/ACSConv.git /tmp/pip-req-build-kmv24qus
  Resolved https://github.com/M3DV/ACSConv.git to commit 95dc860a77e309f010a3d8be1f675e77c7dfeda4
  Preparing metadata (setup.py) ... done


In [7]:
!pip install medmnist

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install tensorboardX

Defaulting to user installation because normal site-packages is not writeable


In [9]:
# Clone the repo to fetch the files
!git clone https://github.com/MedMNIST/experiments.git

fatal: destination path 'experiments' already exists and is not an empty directory.


#### PART B : For all 2D datasets

In [4]:
# Define a list of dataset flags
dataset_flags = ["pathmnist", "octmnist", "pneumoniamnist",
                 "dermamnist", "retinamnist", "breastmnist",
                 "bloodmnist", "tissuemnist", "organamnist",
                 "organcmnist", "organsmnist"]

In [5]:
# Directory containing the log files
base_directory = 'experiments/MedMNIST2D'

# Loop through each dataset flag
for dataset_flag in dataset_flags:
    # Loop through each run for each dataset
    for run in range(5):
        output_folder = f"./MedMNIST2D/{dataset_flag}/run_{run}"
        command = f"python experiments/MedMNIST2D/train_and_eval_pytorch.py --data_flag {dataset_flag} --num_epochs 3 --download --model_flag resnet50 --output_root {output_folder}"
        !{command}

==> Preparing data...
Using downloaded and verified file: /home/atricham/.medmnist/pathmnist.npz
Using downloaded and verified file: /home/atricham/.medmnist/pathmnist.npz
Using downloaded and verified file: /home/atricham/.medmnist/pathmnist.npz
==> Building and training model...
  0%|                                                     | 0/3 [00:00<?, ?it/s]cur_best_auc: 0.984834899409294
cur_best_epoch 0
 33%|███████████████                              | 1/3 [00:50<01:41, 50.73s/it]cur_best_auc: 0.9858806012916779
cur_best_epoch 1
 67%|██████████████████████████████               | 2/3 [01:40<00:50, 50.38s/it]cur_best_auc: 0.9975459930352589
cur_best_epoch 2
100%|█████████████████████████████████████████████| 3/3 [02:31<00:00, 50.40s/it]
pathmnist
train  auc: 0.99769  acc: 0.93347
val  auc: 0.99755  acc: 0.93133
test  auc: 0.97182  acc: 0.77577

==> Preparing data...
Using downloaded and verified file: /home/atricham/.medmnist/pathmnist.npz
Using downloaded and verified file: /home

In [10]:
!ls

BMEhw2.ipynb  MedMNIST2D       MedMNIST3D  Untitled.ipynb
experiments   MedMNIST2D_data  ondemand


In [29]:
# Directory containing the log files
base_directory = 'MedMNIST2D'

# Initialize an empty list to store log file paths
log_file_paths = []

# Recursively search for log files
for root, _, files in os.walk(base_directory):
    for file in files:
        if file.endswith('_log.txt'):
            log_file_paths.append(os.path.join(root, file))

# Initialize an empty list to store extracted data
all_data = []

# Loop through each log file path
for log_file_path in log_file_paths:
    # Extract the dataset name and run information from the path
    parts = log_file_path.split(os.path.sep)
    dataset_name = parts[-3]
    run_info = parts[-4]

    # Check if the log file exists
    if os.path.exists(log_file_path):
        # Read the contents of the log file
        with open(log_file_path, 'r') as file:
            lines = file.readlines()

        # Initialize dictionaries to store metrics for each dataset
        dataset_metrics = {'Run': [], 'Set': [], 'AUC': [], 'Accuracy': [],  'Dataset': []}

        # Process each line (starting from the second line) and extract the relevant information
        for line in lines[1:]:
            parts = line.split()
            if len(parts) == 5 and parts[0] in ['train', 'val', 'test']:
                dataset_metrics['Run'].append(run_info)
                dataset_metrics['Set'].append(parts[0])
                dataset_metrics['AUC'].append(float(parts[2]))
                dataset_metrics['Accuracy'].append(float(parts[4]))
                dataset_metrics['Dataset'].append(dataset_name)

        # Append the extracted data for this log file to the list
        all_data.append(pd.DataFrame(dataset_metrics))
    else:
        print(f"Log file not found: {log_file_path}")

# Concatenate all dataframes into a single dataframe if there is data
if all_data:
    final_df_2d = pd.concat(all_data, ignore_index=True)
    final_df_2d = final_df_2d.sort_values(by=['Run'])

    # Calculate mean and standard deviation for 'train', 'val', and 'test' sets across all runs
    grouped = final_df_2d.groupby(['Dataset', 'Set'])  # Group by both Dataset and Set
    result_dfs = []
    for (dataset, name), group in grouped:
        set_mean = group[['AUC', 'Accuracy']].mean()
        set_std = group[['AUC', 'Accuracy']].std()

        result_df = pd.DataFrame({
            'Dataset': [dataset],
            'Set': [name],
            'Mean AUC': [set_mean['AUC']],
            'Mean Accuracy': [set_mean['Accuracy']],
            'STD AUC': [set_std['AUC']],
            'STD Accuracy': [set_std['Accuracy']]
        })
        result_dfs.append(result_df)

    result_df = pd.concat(result_dfs, ignore_index=True)
    print(result_df)
else:
    print("No log files were processed.")






           Dataset    Set  Mean AUC  Mean Accuracy   STD AUC  STD Accuracy
0       bloodmnist   test  0.981370       0.804676  0.005499      0.048365
1       bloodmnist  train  0.984998       0.817710  0.004530      0.047166
2       bloodmnist    val  0.983504       0.810162  0.003997      0.051472
3      breastmnist   test  0.628196       0.723078  0.040943      0.017200
4      breastmnist  train  0.649840       0.715752  0.037177      0.033581
5      breastmnist    val  0.615706       0.720514  0.057845      0.022933
6       dermamnist   test  0.858866       0.685984  0.007329      0.016042
7       dermamnist  train  0.867070       0.693448  0.008486      0.013131
8       dermamnist    val  0.880814       0.701694  0.004309      0.016371
9         octmnist   test  0.956398       0.754400  0.007887      0.025540
10        octmnist  train  0.968640       0.903648  0.003580      0.008763
11        octmnist    val  0.964722       0.897636  0.002253      0.007885
12     organamnist   test

In [6]:
data_flags = ['nodulemnist3d','adrenalmnist3d','fracturemnist3d','vesselmnist3d','synapsemnist3d']

In [7]:
# Directory containing the log files
base_directory = 'experiments/MedMNIST3D'

# Loop through each dataset flag
for dataset_flag in data_flags:
    # Loop through each run for each dataset
    for run in range(5):
        output_folder = f"./MedMNIST3D/{dataset_flag}/run_{run}"
        command = f"python experiments/MedMNIST3D/train_and_eval_pytorch.py --data_flag {dataset_flag} --num_epochs 3 --download --model_flag resnet50 --output_root {output_folder}"
        !{command}

The ``converters`` are currently experimental. It may not support operations including (but not limited to) Functions in ``torch.nn.functional`` that involved data dimension
==> Preparing data...
100%|███████████████████████████| 29299364/29299364 [00:47<00:00, 611743.13it/s]
Using downloaded and verified file: /home/atricham/.medmnist/nodulemnist3d.npz
Using downloaded and verified file: /home/atricham/.medmnist/nodulemnist3d.npz
Using downloaded and verified file: /home/atricham/.medmnist/nodulemnist3d.npz
==> Building and training model...
  0%|                                                     | 0/3 [00:00<?, ?it/s]cur_best_auc: 0.6504065040650407
cur_best_epoch 0
 33%|███████████████                              | 1/3 [00:15<00:31, 15.93s/it]cur_best_auc: 0.8828881145954317
cur_best_epoch 1
100%|█████████████████████████████████████████████| 3/3 [00:46<00:00, 15.34s/it]
nodulemnist3d
train  auc: 0.83790  acc: 0.82211
val  auc: 0.88289  acc: 0.83636
test  auc: 0.80227  acc: 0.841

In [35]:
# Directory containing the log files
base_directory_3d = 'MedMNIST3D'

# Initialize an empty list to store log file paths
log_file_paths_3d = []

# Recursively search for log files
for root, _, files in os.walk(base_directory_3d):
    for file in files:
        if file.endswith('_log.txt'):
            log_file_paths_3d.append(os.path.join(root, file))

# Initialize an empty list to store extracted data
all_data = []

# Loop through each log file path
for log_file_path in log_file_paths_3d:
    # Extract the dataset name and run information from the path
    parts = log_file_path.split(os.path.sep)
    dataset_name = parts[-3]
    run_info = parts[-4]

    # Check if the log file exists
    if os.path.exists(log_file_path):
        # Read the contents of the log file
        with open(log_file_path, 'r') as file:
            lines = file.readlines()

        # Initialize dictionaries to store metrics for each dataset
        dataset_metrics = {'Run': [], 'Set': [], 'AUC': [], 'Accuracy': [],  'Dataset': []}

        # Process each line (starting from the second line) and extract the relevant information
        for line in lines[1:]:
            parts = line.split()
            if len(parts) == 5 and parts[0] in ['train', 'val', 'test']:
                dataset_metrics['Run'].append(run_info)
                dataset_metrics['Set'].append(parts[0])
                dataset_metrics['AUC'].append(float(parts[2]))
                dataset_metrics['Accuracy'].append(float(parts[4]))
                dataset_metrics['Dataset'].append(dataset_name)

        # Append the extracted data for this log file to the list
        all_data.append(pd.DataFrame(dataset_metrics))
    else:
        print(f"Log file not found: {log_file_path}")

# Concatenate all dataframes into a single dataframe if there is data
if all_data:
    final_df_3d = pd.concat(all_data, ignore_index=True)
    final_df_3d = final_df_3d.sort_values(by=['Run'])

    # Calculate mean and standard deviation for 'train', 'val', and 'test' sets across all runs
    grouped = final_df_3d.groupby(['Dataset', 'Set'])  # Group by both Dataset and Set
    results_dfs = []
    for (dataset, name), group in grouped:
        set_mean = group[['AUC', 'Accuracy']].mean()
        set_std = group[['AUC', 'Accuracy']].std()

        results_df = pd.DataFrame({
            'Dataset': [dataset],
            'Set': [name],
            'Mean AUC': [set_mean['AUC']],
            'Mean Accuracy': [set_mean['Accuracy']],
            'STD AUC': [set_std['AUC']],
            'STD Accuracy': [set_std['Accuracy']]
        })
        results_dfs.append(results_df)

    results_df = pd.concat(results_dfs, ignore_index=True)
    print(results_df)
else:
    print("No log files were processed.")






            Dataset    Set  Mean AUC  Mean Accuracy   STD AUC  STD Accuracy
0    adrenalmnist3d   test  0.855884       0.783222  0.025340      0.026696
1    adrenalmnist3d  train  0.836936       0.806062  0.024693      0.043767
2    adrenalmnist3d    val  0.835526       0.800000  0.010402      0.039256
3   fracturemnist3d   test  0.680030       0.472500  0.021073      0.056948
4   fracturemnist3d  train  0.692322       0.477506  0.024042      0.035388
5   fracturemnist3d    val  0.663868       0.458254  0.020060      0.058009
6     nodulemnist3d   test  0.788656       0.832260  0.034232      0.016291
7     nodulemnist3d  train  0.814730       0.819690  0.023942      0.005374
8     nodulemnist3d    val  0.821798       0.836362  0.038708      0.017672
9    synapsemnist3d   test  0.722810       0.623864  0.029004      0.205827
10   synapsemnist3d  train  0.917400       0.706018  0.050556      0.242751
11   synapsemnist3d    val  0.709756       0.606780  0.042980      0.195938
12    vessel

In [36]:
import shutil

folder_to_zip = 'MedMNIST2D'
output_zip_file = 'MedMNIST2D_partb.zip'

shutil.make_archive(output_zip_file, 'zip', folder_to_zip)


'/home/atricham/MedMNIST2D_partb.zip.zip'

In [37]:
import shutil

folder_to_zip = 'MedMNIST3D'
output_zip_file = 'MedMNIST3D_partb.zip'

shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

'/home/atricham/MedMNIST3D_partb.zip.zip'